In [ ]:
import os
import cv2
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from gtts import gTTS
from IPython.display import Audio, display
import speech_recognition as sr
import sounddevice as sd
import wavio
from google.colab import files

# Function for speaking a text
def speak(text):
    tts = gTTS(text=text, lang='en', slow=False)
    output_file = "output.mp3"
    tts.save(output_file)
    display(Audio(output_file, autoplay=True))
    print(f"Speaking: {text}")

# Function for listening to a command from an uploaded audio file
def listen_to_command_from_file(file_path):
    recognizer = sr.Recognizer()
    with sr.AudioFile(file_path) as source:
        audio = recognizer.record(source)

    try:
        command = recognizer.recognize_google(audio)
        print(f"Command received: {command}")
        return command.lower()
    except sr.UnknownValueError:
        print("Sorry, I couldn't understand the audio.")
        return ""
    except sr.RequestError:
        print("Sorry, the speech service is unavailable.")
        return ""

# Function for listening to a command using microphone
def listen_to_command():
    recognizer = sr.Recognizer()
    duration = 5
    fs = 44100

    print("Listening for a command...")
    audio_data = sd.rec(int(duration * fs), samplerate=fs, channels=1, dtype='int16')
    sd.wait()
    wavio.write("command.wav", audio_data, fs, sampwidth=2)

    with sr.AudioFile("command.wav") as source:
        audio = recognizer.record(source)

    try:
        command = recognizer.recognize_google(audio)
        print(f"Command received: {command}")
        return command.lower()
    except sr.UnknownValueError:
        print("Sorry, I couldn't understand the audio.")
        return ""
    except sr.RequestError:
        print("Sorry, the speech service is unavailable.")
        return ""

# Load dataset from the given path
dataset_path = "/content/braille_dataset"
print(f"Dataset path: {dataset_path}")

# Function for loading and processing images
def load_data(dataset_path):
    X = []
    y = []
    allowed_extensions = ['.png', '.jpg', '.jpeg']

    for root, _, files in os.walk(dataset_path):
        for img_file in files:
            img_path = os.path.join(root, img_file)
            if any(img_file.lower().endswith(ext) for ext in allowed_extensions):
                img = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)
                if img is None:
                    print(f"Warning: Could not load image {img_path}")
                    continue

                label = img_file[0].lower()  # Assuming label is the first character of the filename
                img = cv2.resize(img, (28, 28))
                img = img.flatten()
                X.append(img)
                y.append(label)
            else:
                print(f"Skipping non-image file: {img_path}")

    if len(X) == 0:
        print("No images loaded. Please check the dataset path and file structure.")
    else:
        print(f"Loaded {len(X)} images")

    return np.array(X), np.array(y)

# Load the data
X, y = load_data(dataset_path)

# Train the model and evaluate accuracy
if X.shape[0] > 0:
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
    rf = RandomForestClassifier(n_estimators=100, random_state=42)
    rf.fit(X_train, y_train)

    y_pred = rf.predict(X_test)
    accuracy = accuracy_score(y_test, y_pred)
    print(f"Accuracy: {accuracy * 100:.2f}%")

    while True:
        print("Please upload your audio file (e.g., .wav or .mp3) with the command.")
        uploaded = files.upload()

        # Assuming the user uploads an audio file
        if uploaded:
            file_name = list(uploaded.keys())[0]


In [ ]:
!apt-get install -y portaudio19-dev
!pip install pyaudio

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following additional packages will be installed:
  libportaudio2 libportaudiocpp0
Suggested packages:
  portaudio19-doc
The following NEW packages will be installed:
  libportaudio2 libportaudiocpp0 portaudio19-dev
0 upgraded, 3 newly installed, 0 to remove and 49 not upgraded.
Need to get 188 kB of archives.
After this operation, 927 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy/universe amd64 libportaudio2 amd64 19.6.0-1.1 [65.3 kB]
Get:2 http://archive.ubuntu.com/ubuntu jammy/universe amd64 libportaudiocpp0 amd64 19.6.0-1.1 [16.1 kB]
Get:3 http://archive.ubuntu.com/ubuntu jammy/universe amd64 portaudio19-dev amd64 19.6.0-1.1 [106 kB]
Fetched 188 kB in 2s (114 kB/s)
Selecting previously unselected package libportaudio2:amd64.
(Reading database ... 123629 files and directories currently installed.)
Preparing to unpack .../libportaudio2_19.6.0-1.

In [ ]:
!pip install sounddevice

In [ ]:
!pip install pipwin
!pipwin install pyaudio

  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 18.1 MB/s eta 0:00:00
  Created wheel for pipwin: filename=pipwin-0.5.2-py2.py3-none-any.whl size=8766 sha256=ae9822ae0d8b8e7483be6a2bf1e176c31cd7d96333098e50b7904c8e22692f2b
  Stored in directory: /root/.cache/pip/wheels/b4/2c/53/c5a91c548b9f030b592608c24efda23ff966b1cceac6414765
  Created wheel for docopt: filename=docopt-0.6.2-py2.py3-none-any.whl size=13706 sha256=72fc3d61101aba50a33f9358bc57c1b91869a503287c7b57d007258b1716ea0f
  Stored in directory: /root/.cache/pip/wheels/fc/ab/d4/5da2067ac95b36618c629a5f93f809425700506f72c9732fac
  Created wheel for pyjsparser: filename=pyjsparser-2.7.1-py3-none-any.whl size=25983 sha256=d1c24a72c64493584f4efe216ff16524d6f385932cf9192ef09bd44d316410e7
  Stored in directory: /root/.cache/pip/wheels/5e/81/26/5956478df303e2bf5a85a5df595bb307bd25948a4bab69f7c7
Successfully b

In [ ]:
!pip install wavio

In [ ]:
!pip install kaggle
from google.colab import files
files.upload()

import os
os.environ['KAGGLE_CONFIG_DIR'] = "/content"

!kaggle datasets download -d shanks0465/braille-character-dataset
!unzip braille-character-dataset.zip -d /content/braille_dataset


In [ ]:
!pip install gtts

In [ ]:
!pip install SpeechRecognition

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 32.8/32.8 MB 26.1 MB/s eta 0:00:00


In [ ]:
!pip install gtts